In [2]:
import pandas as pd

# load data
female_lfp = pd.read_csv('data/female-labor-force-participation-rates.csv')

# Rename column headers
female_lfp.rename(columns={'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)': 'Rate', 'Entity': 'Location'}, inplace=True)

# Display first five rows 
female_lfp.head()



,Location,Code,Year,Rate
0,Afghanistan,AFG,1991,15.635
1,Afghanistan,AFG,1992,15.698
2,Afghanistan,AFG,1993,15.715
3,Afghanistan,AFG,1994,15.726
4,Afghanistan,AFG,1995,15.796


In [3]:
# Remove duplicate rows
u_female_lfp = female_lfp.drop_duplicates()

#count of rows in original data frame
row_count = len(female_lfp)

# count of rows in data frame with duplicates dropped
u_row_count = len(u_female_lfp)

# comparing the two row counts to see if there were any duplicates removed.
print(f"Number of rows: {row_count}"  f"\nNumber of unique rows: {u_row_count}")

Number of rows: 6431
Number of unique rows: 6431


In [4]:
#change data types
ud_female_lfp = u_female_lfp.convert_dtypes()


#compare data types
print(f"Old Data Types: \n{u_female_lfp.dtypes}\n")
print(f"New Data Types: \n{ud_female_lfp.dtypes}")

Old Data Types: 
Location     object
Code         object
Year          int64
Rate        float64
dtype: object

New Data Types: 
Location    string[python]
Code        string[python]
Year                 Int64
Rate               Float64
dtype: object


In [5]:
#identify null values
null_rows = ud_female_lfp[ud_female_lfp.isna().any(axis=1)]

#show count of null values per column
print(ud_female_lfp.isnull().sum()) 

#export null values to csv for review
null_rows.to_csv("data/nullrows.csv")




Location      0
Code        416
Year          0
Rate          0
dtype: int64


In [6]:
#filter by null values in the Code column
null_rows = ud_female_lfp[pd.isnull(ud_female_lfp["Code"])]

# Display first five rows 
null_rows.head()

,Location,Code,Year,Rate
1536,East Asia and Pacific (WB),<NA>,1991,66.366035
1537,East Asia and Pacific (WB),<NA>,1992,66.24243
1538,East Asia and Pacific (WB),<NA>,1993,65.92144
1539,East Asia and Pacific (WB),<NA>,1994,65.88117
1540,East Asia and Pacific (WB),<NA>,1995,65.73217


In [7]:
#replace  na values with location value
ud_female_lfp['Code'] = ud_female_lfp['Code'].fillna(ud_female_lfp['Location'])

#check for null values again
null_rows = ud_female_lfp[pd.isnull(ud_female_lfp["Code"])]

# Display first five rows 
null_rows.head()

#yay no null values!

,Location,Code,Year,Rate


In [8]:
#save to csv
ud_female_lfp.to_csv("data/participation_clean.csv")

In [9]:
# let's load another piece of data
continents = pd.read_csv('data/continents-according-to-our-world-in-data.csv')

# Rename column headers
continents.rename(columns={'World regions according to OWID': 'Continent', 'Entity': 'Location'}, inplace=True)

# Display first five rows 
continents.head()

,Location,Code,Year,Continent
0,Afghanistan,AFG,2023,Asia
1,Aland Islands,ALA,2023,Europe
2,Albania,ALB,2023,Europe
3,Algeria,DZA,2023,Africa
4,American Samoa,ASM,2023,Oceania


In [10]:
# Remove duplicate rows
u_continents = continents.drop_duplicates()

#count of rows in original data frame
row_count = len(continents)

# count of rows in data frame with duplicates dropped
u_row_count = len(u_continents)

# comparing the two row counts to see if there were any duplicates removed.
print(f"Number of rows: {row_count}"  f"\nNumber of unique rows: {u_row_count}")

Number of rows: 271
Number of unique rows: 271


In [11]:
#change data types
ud_continents = u_continents.convert_dtypes()


#compare data types
print(f"Old Data Types: \n{u_continents.dtypes}\n")
print(f"New Data Types: \n{ud_continents.dtypes}")

Old Data Types: 
Location     object
Code         object
Year          int64
Continent    object
dtype: object

New Data Types: 
Location     string[python]
Code         string[python]
Year                  Int64
Continent    string[python]
dtype: object


In [12]:
#identify null values
null_rows_continents = ud_continents[ud_continents.isna().any(axis=1)]

#show count of null values per column
print(ud_continents.isnull().sum()) 

#export null values to csv for review
null_rows_continents.to_csv("data/nullrowscontinents.csv")




Location      0
Code         14
Year          0
Continent     0
dtype: int64


In [13]:
#filter by null values in the Code column
null_rows = ud_continents[pd.isnull(ud_continents["Code"])]

# Display first five rows 
null_rows.head()

,Location,Code,Year,Continent
61,Democratic Republic of Vietnam,<NA>,2023,Asia
66,Duchy of Modena and Reggio,<NA>,2023,Europe
67,Duchy of Parma and Piacenza,<NA>,2023,Europe
81,Federal Republic of Central America,<NA>,2023,North America
93,Grand Duchy of Baden,<NA>,2023,Europe


In [14]:
#drop null values
ud_continents.dropna()

,Location,Code,Year,Continent
0,Afghanistan,AFG,2023,Asia
1,Aland Islands,ALA,2023,Europe
2,Albania,ALB,2023,Europe
3,Algeria,DZA,2023,Africa
4,American Samoa,ASM,2023,Oceania
...,...,...,...,...
266,Yemen Arab Republic,OWID_YAR,2023,Asia
267,Yemen People's Republic,OWID_YPR,2023,Asia
268,Yugoslavia,OWID_YGS,2023,Europe
269,Zambia,ZMB,2023,Africa


In [15]:
#save to csv
ud_continents.to_csv("data/continents_clean.csv")


In [22]:
import sqlite3
import csv

#setup database
with sqlite3.connect("women_in_the_workforce.db", timeout=30) as con:
    #creates the tables
    ud_female_lfp.to_sql('participation', con, if_exists='replace', index=False)
    ud_continents.to_sql('continents', con, if_exists='replace', index=False)

cur = con.cursor()


cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall()) 

[('labor_force_participation_rates2',), ('continents2',), ('labor_force_participation_rates',), ('avgbycontinent',), ('participation',), ('continents',)]
